In [1]:
# %%
# ================================
# Cell 1 — Imports & Configuration
# ================================
import os, math, random, time, platform
import numpy as np
import pandas as pd
from PIL import Image
from collections import Counter

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torchvision import transforms
import torchvision

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, f1_score, roc_auc_score, confusion_matrix,
    classification_report, matthews_corrcoef
)

# ---- Config (edit these two to match your environment) ----
CSV_PATH  = r"C:\Users\PC\Desktop\final project\CBIS-DDSM\mass\data\yolo_crops.csv"  # full path to your CSV
ROOT_DIR  = r"C:\Users\PC\Desktop\final project\CBIS-DDSM\mass"

# Training hyperparameters
IMG_SIZE     = 384
BATCH_SIZE   = 16
EPOCHS       = 25
LR           = 2e-4
WEIGHT_DECAY = 1e-4
OUT_PATH     = "resnet50_cbis.pt"   # saved best checkpoint (by AUC)
USE_CLAHE    = True                 # set False to disable CLAHE (val too)

# Reproducibility
SEED = 42
random.seed(SEED); np.random.seed(SEED)
torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD  = [0.229, 0.224, 0.225]

print("Torch:", torch.__version__, "| TorchVision:", torchvision.__version__)
print("CUDA available:", torch.cuda.is_available(), "| Device:", DEVICE)
print("CSV_PATH:", CSV_PATH)
print("ROOT_DIR:", ROOT_DIR)
assert os.path.exists(CSV_PATH), "CSV file not found!"

# %%
# =====================================
# Cell 2 — Load CSV & Basic Sanity Check
# =====================================
df = pd.read_csv(CSV_PATH)

required_cols = {"yolo_cropped_image_path", "label",
                 "laterality_LEFT","laterality_RIGHT","view_CC","view_MLO"}
missing = required_cols - set(df.columns)
assert not missing, f"Missing columns in CSV: {missing}"

def _exists(rel):
    p = str(rel).replace("\\", os.sep)
    if ROOT_DIR and not os.path.isabs(p):
        p = os.path.join(ROOT_DIR, p)
    return os.path.exists(p)

df["exists"] = df["yolo_cropped_image_path"].apply(_exists)
df = df[df["exists"]].copy()
df["label"] = df["label"].astype(int)

print("Rows (after existence filter):", len(df))
print(df["label"].value_counts().rename({0:"benign",1:"malignant"}).to_string())
assert len(df) > 0, "No valid rows after existence filter."

# %%
# ============================
# Cell 3 — Transforms (Fixed)
# ============================
try:
    import albumentations as A
    from albumentations.pytorch import ToTensorV2
    import cv2
    _HAS_ALBU = True
    print("Albumentations available.")
except Exception:
    _HAS_ALBU = False
    print("Albumentations NOT available. Falling back to torchvision transforms.")

def build_transforms(img_size=IMG_SIZE, use_clahe=USE_CLAHE):
    if _HAS_ALBU:
        BLACK3 = (0, 0, 0)  # required when using BORDER_CONSTANT
        aug_train = A.Compose([
            (A.CLAHE(clip_limit=3.0, tile_grid_size=(8,8), p=1.0) if use_clahe else A.NoOp()),
            A.GaussianBlur(blur_limit=(3,5), p=0.2),
            A.MotionBlur(blur_limit=3, p=0.1),
            A.RandomBrightnessContrast(0.15, 0.25, p=0.6),
            A.RandomResizedCrop(size=(img_size, img_size), scale=(0.9, 1.0), ratio=(0.9, 1.1), p=1.0),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.ShiftScaleRotate(shift_limit=0.02, scale_limit=0.10, rotate_limit=20,
                               border_mode=cv2.BORDER_CONSTANT, value=BLACK3, p=0.6),
            A.Downscale(scale_min=0.90, scale_max=0.95, p=0.2),
            A.GaussNoise(var_limit=(5.0, 20.0), p=0.2),
            A.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
            ToTensorV2(),
        ])
        aug_val = A.Compose([
            (A.CLAHE(clip_limit=3.0, tile_grid_size=(8,8), p=1.0) if use_clahe else A.NoOp()),
            A.LongestMaxSize(max_size=img_size),
            A.PadIfNeeded(min_height=img_size, min_width=img_size,
                          border_mode=cv2.BORDER_CONSTANT, value=BLACK3, p=1.0),
            A.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
            ToTensorV2(),
        ])
    else:
        aug_train = transforms.Compose([
            transforms.Grayscale(num_output_channels=3),
            transforms.Resize((img_size, img_size)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomVerticalFlip(),
            transforms.RandomRotation(20),
            transforms.ToTensor(),
            transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
        ])
        aug_val = transforms.Compose([
            transforms.Grayscale(num_output_channels=3),
            transforms.Resize((img_size, img_size)),
            transforms.ToTensor(),
            transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
        ])
    return aug_train, aug_val

t_train, t_val = build_transforms()
print("Transforms ready.")

# quick validation
sample_path = os.path.join(ROOT_DIR, str(df.iloc[0]["yolo_cropped_image_path"]).replace("\\", os.sep))
img = Image.open(sample_path).convert("L")
if _HAS_ALBU:
    arr = np.stack([np.array(img)]*3, axis=2).astype(np.uint8)
    x = t_val(image=arr)["image"]
else:
    x = t_val(img)
print("One sample tensor shape:", tuple(x.shape))  # (3, IMG_SIZE, IMG_SIZE)
assert x.shape[0] == 3 and x.shape[1] == IMG_SIZE and x.shape[2] == IMG_SIZE, "Transform shape mismatch!"

# %%
# ===================================
# Cell 4 — Dataset & DataLoaders
# ===================================
IS_WIN = (os.name == "nt") or ("windows" in platform.system().lower())
NUM_WORKERS = 0 if IS_WIN else 4
PIN_MEMORY = torch.cuda.is_available()

class CropsDataset(Dataset):
    def __init__(self, df, root_dir=ROOT_DIR, transform=None, path_col="yolo_cropped_image_path", label_col="label"):
        self.df = df.reset_index(drop=True)
        self.root = root_dir
        self.t = transform
        self.path_col = path_col
        self.label_col = label_col

    def __len__(self): return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        rel = str(row[self.path_col]).replace("\\", os.sep)
        p = os.path.join(self.root, rel) if self.root and not os.path.isabs(rel) else rel

        im = Image.open(p).convert("L")
        if _HAS_ALBU:
            arr = np.array(im)
            arr3 = np.stack([arr, arr, arr], axis=2)  # (H, W, 3)
            x = self.t(image=arr3)["image"]
        else:
            x = self.t(im)

        y = torch.tensor(int(row[self.label_col]), dtype=torch.float32)
        meta = torch.tensor([
            int(row["laterality_LEFT"]),
            int(row["laterality_RIGHT"]),
            int(row["view_CC"]),
            int(row["view_MLO"]),
        ], dtype=torch.float32)

        return x, y, meta

train_df, val_df = train_test_split(df, test_size=0.2, stratify=df["label"], random_state=SEED)
train_ds = CropsDataset(train_df, transform=t_train)
val_ds   = CropsDataset(val_df,   transform=t_val)

# Weighted sampler for class imbalance
class_counts = Counter(train_df["label"].tolist())
class_weights = {c: len(train_df)/class_counts[c] for c in class_counts}
sample_weights = torch.tensor(train_df["label"].map(class_weights).values, dtype=torch.float32)
sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)

train_loader = DataLoader(
    train_ds, batch_size=BATCH_SIZE, sampler=sampler,
    num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY, persistent_workers=(NUM_WORKERS > 0)
)
val_loader = DataLoader(
    val_ds, batch_size=BATCH_SIZE*2, shuffle=False,
    num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY, persistent_workers=(NUM_WORKERS > 0)
)

xb, yb, mb = next(iter(train_loader))
print(f"OS: {'Windows' if IS_WIN else platform.system()} | num_workers={NUM_WORKERS} | pin_memory={PIN_MEMORY}")
print("Train batch:", xb.shape, yb.shape, mb.shape, "| pos in batch:", int(yb.sum().item()))
assert xb.ndim == 4 and xb.shape[1] == 3
assert yb.ndim == 1 and mb.shape[1] == 4


# ============================
# Cell 5 — Model Definition
# ============================
class ResNetWithMeta(nn.Module):
    def __init__(self, meta_dim=4):
        super().__init__()
        base = torchvision.models.resnet50(weights=torchvision.models.ResNet50_Weights.IMAGENET1K_V2)
        # backbone up to (and including) avgpool, without fc
        self.backbone = nn.Sequential(*list(base.children())[:-1])  # (B, 2048, 1, 1)
        self.in_feat = base.fc.in_features  # 2048
        self.meta_bn = nn.BatchNorm1d(meta_dim)
        self.head = nn.Sequential(
            nn.Linear(self.in_feat + meta_dim, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.4),
            nn.Linear(512, 1),
        )

    def forward(self, x, meta):
        feats = self.backbone(x).squeeze(-1).squeeze(-1)  # (B, 2048)
        meta = self.meta_bn(meta)                         # (B, 4)
        fused = torch.cat([feats, meta], dim=1)           # (B, 2052)
        return self.head(fused)                           # (B, 1)

def make_model():
    return ResNetWithMeta(meta_dim=4)

model = make_model().to(DEVICE)

with torch.no_grad():
    out = model(xb.to(DEVICE), mb.to(DEVICE))
print("Model output shape:", tuple(out.shape))
assert out.shape[1] == 1


# ============================
# Cell 6 — Loss, Optimizer, LR
# ============================
def focal_loss(inputs, targets, alpha=0.8, gamma=2.0, reduction='mean'):
    bce = nn.functional.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
    probs = torch.sigmoid(inputs)
    p_t = probs*targets + (1-probs)*(1-targets)
    loss = (alpha * (1-p_t)**gamma) * bce
    return loss.mean() if reduction=='mean' else loss.sum()

criterion_mix = lambda logits, y: 0.5*focal_loss(logits, y) + 0.5*nn.functional.binary_cross_entropy_with_logits(logits, y)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

# tiny opt step sanity
model.train()
x_small = xb[:4].to(DEVICE); y_small = yb[:4].to(DEVICE).unsqueeze(1); m_small = mb[:4].to(DEVICE)
optimizer.zero_grad()
loss = criterion_mix(model(x_small, m_small), y_small)
loss.backward(); optimizer.step()
print("One tiny opt step OK. Loss:", float(loss.item()))


# ==================================
# Cell 7 — Train & Evaluation Utils
# ==================================
@torch.no_grad()
def evaluate(model, loader):
    model.eval()
    y_true, y_prob = [], []
    for x, y, m in loader:
        x = x.to(DEVICE); m = m.to(DEVICE)
        logits = model(x, m)
        probs = torch.sigmoid(logits).squeeze(1).cpu().numpy().tolist()
        y_prob += probs
        y_true += y.numpy().tolist()
    y_pred = [1 if p >= 0.5 else 0 for p in y_prob]
    acc = accuracy_score(y_true, y_pred)
    f1  = f1_score(y_true, y_pred)
    auc = roc_auc_score(y_true, y_prob) if len(set(y_true)) > 1 else float("nan")
    return acc, f1, auc, (y_true, y_prob, y_pred)

def train_epoch(model, loader, optimizer, criterion):
    model.train()
    tot = 0.0
    for x, y, m in loader:
        x = x.to(DEVICE); y = y.to(DEVICE).unsqueeze(1); m = m.to(DEVICE)
        optimizer.zero_grad()
        logits = model(x, m)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()
        tot += loss.item() * x.size(0)
    return tot / len(loader.dataset)

# Dry eval
acc, f1, auc, _ = evaluate(model, val_loader)
print(f"Dry evaluation OK — acc={acc:.3f}, f1={f1:.3f}, auc={auc:.3f}")

# =======================
# Cell 8 — Train the Model
# =======================
best_auc = -1.0
history = []

for epoch in range(1, EPOCHS+1):
    tr_loss = train_epoch(model, train_loader, optimizer, criterion_mix)
    acc, f1, auc, _ = evaluate(model, val_loader)
    scheduler.step()

    history.append({"epoch": epoch, "train_loss": tr_loss, "val_acc": acc, "val_f1": f1, "val_auc": auc})
    print(f"Epoch {epoch:02d}: loss={tr_loss:.4f} | acc={acc:.4f} | f1={f1:.4f} | auc={auc:.4f}")

    if auc > best_auc:
        best_auc = auc
        torch.save({
            "model": model.state_dict(),
            "img_size": IMG_SIZE,
            "mean": IMAGENET_MEAN,
            "std": IMAGENET_STD
        }, OUT_PATH)

print("Best AUC:", best_auc, "| Saved:", OUT_PATH)
assert os.path.exists(OUT_PATH), "Checkpoint was not saved."


# ==========================================
# Cell 9 — Final Validation & Threshold Tuning (Unified Style)
# ==========================================
from sklearn.metrics import (
    accuracy_score, f1_score, roc_auc_score, confusion_matrix,
    classification_report, matthews_corrcoef
)
import numpy as np

ckpt = torch.load(OUT_PATH, map_location=DEVICE)
model_best = make_model().to(DEVICE)
model_best.load_state_dict(ckpt["model"])
model_best.eval()

# Eval @0.5
acc, f1, auc, (y_true, y_prob, y_pred) = evaluate(model_best, val_loader)
print(f"Best ckpt @0.5 — acc={acc:.4f}, f1={f1:.4f}, auc={auc:.4f}")

# Threshold sweep to maximize F1
best_t, best_f1 = 0.5, f1
for t in np.linspace(0.1, 0.9, 50):
    yp = [1 if p >= t else 0 for p in y_prob]
    f1_t = f1_score(y_true, yp)
    if f1_t > best_f1:
        best_f1, best_t = f1_t, t

yp_opt = [1 if p >= best_t else 0 for p in y_prob]
cm = confusion_matrix(y_true, yp_opt)
tn, fp, fn, tp = cm.ravel()

sensitivity = tp / (tp + fn + 1e-8)
specificity = tn / (tn + fp + 1e-8)
precision   = tp / (tp + fp + 1e-8)
npv         = tn / (tn + fn + 1e-8)
balanced_acc= 0.5 * (sensitivity + specificity)
mcc         = matthews_corrcoef(y_true, yp_opt)

print("\n=== Metrics at optimal threshold ===")
print(f"Threshold        : {best_t:.3f}")
print(f"Accuracy         : {accuracy_score(y_true, yp_opt):.4f}")
print(f"Balanced Acc     : {balanced_acc:.4f}")
print(f"Sensitivity      : {sensitivity:.4f}")
print(f"Specificity      : {specificity:.4f}")
print(f"Precision        : {precision:.4f}")
print(f"NPV              : {npv:.4f}")
print(f"F1 Score         : {best_f1:.4f}")
print(f"AUC              : {roc_auc_score(y_true, y_prob):.4f}")
print(f"MCC              : {mcc:.4f}")

print("\nConfusion matrix:\n", cm)
print("\nClassification Report:\n",
      classification_report(y_true, yp_opt, target_names=["benign","malignant"]))


# =======================================
# Cell 10 — Grad-CAM Utilities & Example
# =======================================
import torch.nn.functional as F
import cv2

def preprocess_gray3(path, size=IMG_SIZE):
    im = Image.open(path).convert("L").resize((size,size))
    im = np.array(im)
    im = np.stack([im, im, im], axis=2)
    t = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD)
    ])
    x = t(Image.fromarray(im))
    return x.unsqueeze(0).to(DEVICE)

class GradCAM:
    """
    Works with the ResNetWithMeta wrapper. Hooks the 'layer4' block inside the backbone.
    Since backbone is a Sequential of ResNet children (without fc), 'layer4' sits at index 7.
    """
    def __init__(self, model, target_layer_idx=7):
        self.model = model.eval()
        self.gradients = None
        self.activations = None
        layer = list(self.model.backbone.children())[target_layer_idx]
        layer.register_forward_hook(self._fhook)
        layer.register_full_backward_hook(self._bhook)

    def _fhook(self, m, inp, out): self.activations = out.detach()
    def _bhook(self, m, gin, gout): self.gradients = gout[0].detach()

    @torch.no_grad()
    def predict(self, x, meta):
        logits = self.model(x, meta)
        return torch.sigmoid(logits)[0,0].item()

    def cam(self, x, meta):
        logits = self.model(x, meta)
        self.model.zero_grad()
        logits[0,0].backward()
        g = self.gradients
        a = self.activations
        w = torch.mean(g, dim=(2,3), keepdim=True)
        cam = (w * a).sum(dim=1)
        cam = F.relu(cam)[0].cpu().numpy()
        cam = cv2.resize(cam, (x.shape[-1], x.shape[-2]))
        cam = (cam - cam.min()) / (cam.max() + 1e-6)
        return cam

val_sample_path = os.path.join(ROOT_DIR, str(val_df.iloc[0]["yolo_cropped_image_path"]).replace("\\", os.sep))
x = preprocess_gray3(val_sample_path, size=IMG_SIZE)
row0 = val_df.iloc[0]
meta0 = torch.tensor([
    float(row0["laterality_LEFT"]), float(row0["laterality_RIGHT"]),
    float(row0["view_CC"]), float(row0["view_MLO"])
], dtype=torch.float32, device=DEVICE).unsqueeze(0)

gc = GradCAM(model_best, target_layer_idx=7)
prob = gc.predict(x, meta0)
heat = gc.cam(x, meta0)
print("Predicted malignant probability:", prob)
print("Grad-CAM heatmap shape:", heat.shape, "min/max:", float(heat.min()), float(heat.max()))
assert heat.ndim == 2


Torch: 2.5.1 | TorchVision: 0.20.1
CUDA available: True | Device: cuda
CSV_PATH: C:\Users\PC\Desktop\final project\CBIS-DDSM\mass\data\yolo_crops.csv
ROOT_DIR: C:\Users\PC\Desktop\final project\CBIS-DDSM\mass
Rows (after existence filter): 1574
label
benign       799
malignant    775


INFO:albumentations.check_version:A new version of Albumentations is available: 2.0.8 (you have 1.4.7). Upgrade using: pip install --upgrade albumentations


Albumentations available.
Transforms ready.
One sample tensor shape: (3, 384, 384)
OS: Windows | num_workers=0 | pin_memory=True
Train batch: torch.Size([16, 3, 384, 384]) torch.Size([16]) torch.Size([16, 4]) | pos in batch: 9
Model output shape: (16, 1)
One tiny opt step OK. Loss: 0.4323744475841522
Dry evaluation OK — acc=0.619, f1=0.620, auc=0.618
Epoch 01: loss=0.4133 | acc=0.5619 | f1=0.5633 | auc=0.5751
Epoch 02: loss=0.4039 | acc=0.5905 | f1=0.4735 | auc=0.5829
Epoch 03: loss=0.4073 | acc=0.5397 | f1=0.6272 | auc=0.6156
Epoch 04: loss=0.4000 | acc=0.5651 | f1=0.5785 | auc=0.6127
Epoch 05: loss=0.4021 | acc=0.6127 | f1=0.6090 | auc=0.6477
Epoch 06: loss=0.4011 | acc=0.6190 | f1=0.6154 | auc=0.6542
Epoch 07: loss=0.3939 | acc=0.5746 | f1=0.5379 | auc=0.5921
Epoch 08: loss=0.3809 | acc=0.6159 | f1=0.6388 | auc=0.6710
Epoch 09: loss=0.3867 | acc=0.6825 | f1=0.6350 | auc=0.7424
Epoch 10: loss=0.3726 | acc=0.6286 | f1=0.5483 | auc=0.6977
Epoch 11: loss=0.3731 | acc=0.6571 | f1=0.6538 

C:\Users\PC\AppData\Local\Temp\ipykernel_6924\3746674740.py:345: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(OUT_PATH, map_location=DEVICE)


Best ckpt @0.5 — acc=0.6921, f1=0.6689, auc=0.7629

=== Metrics at optimal threshold ===
Threshold        : 0.329
Accuracy         : 0.6667
Balanced Acc     : 0.6693
Sensitivity      : 0.8323
Specificity      : 0.5062
Precision        : 0.6202
NPV              : 0.7570
F1 Score         : 0.7107
AUC              : 0.7629
MCC              : 0.3573

Confusion matrix:
 [[ 81  79]
 [ 26 129]]

Classification Report:
               precision    recall  f1-score   support

      benign       0.76      0.51      0.61       160
   malignant       0.62      0.83      0.71       155

    accuracy                           0.67       315
   macro avg       0.69      0.67      0.66       315
weighted avg       0.69      0.67      0.66       315

Predicted malignant probability: 0.18509960174560547
Grad-CAM heatmap shape: (384, 384) min/max: 0.0 0.9999644160270691
